In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

In [7]:
class BFgen(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, output_size, n_layers=1):
        super(BFgen, self).__init__()
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()
        
    def forward(self, input_token, (hidden, cell)):
        embeds = self.encoder(input_token)
        output, (hidden, cell) = self.lstm(
            embeds.view(len(input_token), 1, -1), (hidden, cell))
        output = self.decoder(output.view(len(input_token), -1))
        output = self.softmax(output)
        return output, (hidden, cell)
    
    def init_hidden_zero(self):
        return Variable(torch.zero(1, self.hidden_size))
    
    def init_hidden_normal(self):
        means = torch.zeros(1, 10)
        std = torch.Tensor([0.001]*self.hidden_size).unsqueeze(0)
        return Variable(torch.normal(means, std))

In [6]:
embedding_size = 10
hidden_size = 35
output_size = 9
n_layers = 2